In [1]:
import sys
import os

# 경로 설정: 스크립트 경로에서 상위 디렉토리로 이동한 후 src 경로 추가
health_data_path = os.path.abspath(os.path.join('..', 'src'))
health_learning_data_path = os.path.abspath(os.path.join(os.getcwd(), "../../HealthModelPipeline/dataflow/src"))
preprocessing_path = os.path.abspath(os.path.join(os.getcwd(), "../../PipelinePrep/src"))

paths = [health_data_path, health_learning_data_path, preprocessing_path]

def add_paths(paths):
    """
    지정된 경로들이 sys.path에 없으면 추가하는 함수.
    
    Parameters:
    - paths (list): 추가하려는 경로들의 리스트.
    """
    for path in paths:
        if path not in sys.path:
            sys.path.append(path)
            print(f"Path added: {path}")
        else:
            print(f"Path already exists: {path}")
            
add_paths(paths)

Path added: C:\Users\admin\Techross\HealthPipeline\src
Path added: C:\Users\admin\Techross\HealthModelPipeline\dataflow\src
Path added: C:\Users\admin\Techross\PipelinePrep\src


In [2]:
# basic
import pandas as pd

# time
import schedule
import time

# module.healthchecker
from stat_healthchecker.total_system_health_algorithm import apply_system_health_algorithms_with_total
from models_healthchecker.total_system_health_learning_algorithm import apply_system_health_learning_algorithms_with_total
from prep.load_processing import distribute_by_application

# module.dataline
from stat_dataline.scheduled_data_fetcher import fetch_data_on_schedule
from stat_dataline.logger_confg import logger
from stat_dataline.select_dataset import get_dataframe_from_database
from stat_dataline.search_status_flag import filter_by_flag_status

In [3]:
def get_latest_date_on_schedule():

    fetched_data = filter_by_flag_status()
    
    # 해당 추출 데이터 그룹화
    grouped_data = fetched_data.groupby(['SHIP_ID','OP_INDEX','SECTION']).count() 
    
    # 그룹 후 인덱스 추출 
    grouped_index = grouped_data.index

    return grouped_index, fetched_data

In [4]:
grouped_index, fetched_data = get_latest_date_on_schedule()

FLAG updated to 1 for the filtered data.
/n[건전성 분석을 위한 날짜 추출]...
[datetime.date(2024, 12, 28), datetime.date(2024, 12, 29)]
/n [건전성 분석 데이터 리턴..]


In [7]:
datas = pd.DataFrame(index = grouped_index).reset_index()

datas[(datas['SHIP_ID']=='T20210706001') & (datas['OP_INDEX']==791) & (datas['SECTION']==0)]

,SHIP_ID,OP_INDEX,SECTION
5326,T20210706001,791,0


In [5]:
def schedule_health_assessment():
    
    grouped_index, fetched_data = get_latest_date_on_schedule()
    
    for index in grouped_index:

        # 해당 오퍼레이션 선박, 인덱스, 섹션 추출
        ship_id =  index[0]
        op_index = index[1]
        section =  index[2] 
        
        # 데이터 처리를 위한 갯수 조건을 만족하는지 판단 
        selected_df = fetched_data[(fetched_data['SHIP_ID']==ship_id) & (fetched_data['OP_INDEX']==op_index) & (fetched_data['SECTION']==section)]

        # 해당 오퍼레이션 데이터 길이 추출
        data_len = len(selected_df)

        # 해당 오퍼레이션 시작 시간 추출
        date_time = selected_df.iloc[0]['DATA_TIME']
       
        print(f'SHIP_ID : {ship_id} / OP_INDEX : {op_index} / SECTION : {section} -  데이터 선택 ({data_len})')
        
        if (data_len>=160) :
            
            print(f'SHIP_ID : {ship_id} / OP_INDEX : {op_index} / SECTION : {section} -  조건 통과')      
            
            try:
                sensor, preprocessed = distribute_by_application(ship_id=ship_id, op_index=op_index, section=section)
                if sensor is None and preprocessed is None:
                    print("선박 데이터 프레임이 존재하지 않습니다.")
                    continue       

                elif preprocessed is not None:
                    logger.info(f'SHIP_ID={ship_id} | OP_INDEX={op_index} | SECTION={section} | START_TIME={date_time} | LOG_ENTRY=The results were derived from the model and statistics package | TYPE=all | IS_PROCESSED=True')
                    print("전처리 후 학습 데이터 프레임이 존재합니다.")
                    apply_system_health_algorithms_with_total(sensor, ship_id, op_index, section)
                    apply_system_health_learning_algorithms_with_total(data=preprocessed, ship_id=ship_id, op_index=op_index, section=section)
                else:
                    logger.info(f'SHIP_ID={ship_id} | OP_INDEX={op_index} | SECTION={section} | START_TIME={date_time}  | LOG_ENTRY=After preprocessing, the model data frame does not exist, so only the statistical algorithm proceeds alone | TYPE=stats | IS_PROCESSED=True')
                    print("전처리 후 모델 데이터 프레임이 존재하지 않아 통계 알고리즘 단독 진행합니다.")
                    apply_system_health_algorithms_with_total(data=sensor, ship_id=ship_id, op_index=op_index, section=section)
                
            except ValueError as e :
                logger.info(f'SHIP_ID={ship_id} | OP_INDEX={op_index} | SECTION={section} | START_TIME={date_time} | LOG_ENTRY={e} | TYPE=exceptional_handling | IS_PROCESSED=False')
                print(f'에러 발생: {e}. 다음 반복으로 넘어갑니다.')
                continue  # 에러 발생 시 다음 반복으로 넘어감\

            except KeyError as e :
                logger.info(f'SHIP_ID={ship_id} | OP_INDEX={op_index} | SECTION={section} | START_TIME={date_time} | LOG_ENTRY={e} | TYPE=exceptional_handling | IS_PROCESSED=False')
                print(f'에러 발생: {e}. 다음 반복으로 넘어갑니다.')

            except TypeError as e :
                logger.info(f'SHIP_ID={ship_id} | OP_INDEX={op_index} | SECTION={section} | START_TIME={date_time} | LOG_ENTRY={e} | TYPE=exceptional_handling | IS_PROCESSED=False')
                print(f'에러 발생: {e}. 다음 반복으로 넘어갑니다.')
                continue  # 에러 발생 시 다음 반복으로 넘어감
                
            except IndexError as e :
                print(f'에러 발생: {e}. 다음 반복으로 넘어갑니다.')
                logger.info(f'SHIP_ID={ship_id} | OP_INDEX={op_index} | SECTION={section} | START_TIME={date_time} | LOG_ENTRY={e} | TYPE=exceptional_handling | IS_PROCESSED=False')
                continue  # 에러 발생 시 다음 반복으로 넘어감 
        else:
            logger.info(f'SHIP_ID={ship_id} | OP_INDEX={op_index} | SECTION={section} | START_TIME={date_time} | LOG_ENTRY=The data length is {data_len} and does not satisfy the condition | TYPE=data_length_limit | IS_PROCESSED=False')

In [ ]:
# 스케줄 설정: 3일에 한 번씩 데이터 가져오기
schedule.every(1).days.at("11:05").do(schedule_health_assessment)
# schedule.every(5).minutes.do(schedule_health_assessment)

# 스케줄 지속 실행
while True:
    print("스케줄 시작")
    schedule.run_pending()
    time.sleep(1)

스케줄 시작
스케줄 시작
스케줄 시작
스케줄 시작
스케줄 시작
스케줄 시작
스케줄 시작
스케줄 시작
스케줄 시작
스케줄 시작
스케줄 시작
스케줄 시작
스케줄 시작
스케줄 시작
스케줄 시작
스케줄 시작
스케줄 시작
스케줄 시작
스케줄 시작
스케줄 시작
스케줄 시작
스케줄 시작
스케줄 시작
스케줄 시작
스케줄 시작
스케줄 시작
스케줄 시작
스케줄 시작
스케줄 시작
스케줄 시작
스케줄 시작
스케줄 시작
스케줄 시작
스케줄 시작
스케줄 시작
스케줄 시작
스케줄 시작
스케줄 시작
스케줄 시작
스케줄 시작
스케줄 시작
스케줄 시작
스케줄 시작
스케줄 시작
스케줄 시작
스케줄 시작
스케줄 시작
스케줄 시작
스케줄 시작
스케줄 시작
스케줄 시작
스케줄 시작
스케줄 시작
스케줄 시작
스케줄 시작
스케줄 시작
스케줄 시작
스케줄 시작
스케줄 시작
스케줄 시작
스케줄 시작
스케줄 시작
스케줄 시작
스케줄 시작
스케줄 시작
스케줄 시작
스케줄 시작
스케줄 시작
스케줄 시작
스케줄 시작
스케줄 시작
스케줄 시작
SHIP_ID : T130422-00101 / OP_INDEX : 59 / SECTION : 0 -  데이터 선택 (8)
SHIP_ID : T130422-00101 / OP_INDEX : 60 / SECTION : 0 -  데이터 선택 (14)
SHIP_ID : T130422-00101 / OP_INDEX : 61 / SECTION : 0 -  데이터 선택 (31)
SHIP_ID : T130509-00201 / OP_INDEX : 545 / SECTION : 0 -  데이터 선택 (40)
SHIP_ID : T130509-00201 / OP_INDEX : 546 / SECTION : 0 -  데이터 선택 (15)
SHIP_ID : T130509-00201 / OP_INDEX : 547 / SECTION : 0 -  데이터 선택 (7)
SHIP_ID : T130509-00201 / OP_INDEX : 548 / SECTION : 0 -  데이터 선택 (48)
SHIP_ID : T

SHIP_ID : T170524-00101 / OP_INDEX : 411 / SECTION : 0 -  데이터 선택 (1)
SHIP_ID : T170524-00101 / OP_INDEX : 412 / SECTION : 0 -  데이터 선택 (1)
SHIP_ID : T170524-00101 / OP_INDEX : 413 / SECTION : 0 -  데이터 선택 (1)
SHIP_ID : T170524-00101 / OP_INDEX : 414 / SECTION : 0 -  데이터 선택 (1)
SHIP_ID : T170524-00101 / OP_INDEX : 415 / SECTION : 0 -  데이터 선택 (1)
SHIP_ID : T170524-00101 / OP_INDEX : 416 / SECTION : 0 -  데이터 선택 (1)
SHIP_ID : T170524-00101 / OP_INDEX : 417 / SECTION : 0 -  데이터 선택 (1)
SHIP_ID : T170524-00101 / OP_INDEX : 418 / SECTION : 0 -  데이터 선택 (1)
SHIP_ID : T170524-00101 / OP_INDEX : 419 / SECTION : 0 -  데이터 선택 (1)
SHIP_ID : T170524-00101 / OP_INDEX : 420 / SECTION : 0 -  데이터 선택 (2)
SHIP_ID : T170524-00101 / OP_INDEX : 421 / SECTION : 0 -  데이터 선택 (1)
SHIP_ID : T170524-00101 / OP_INDEX : 422 / SECTION : 0 -  데이터 선택 (1)
SHIP_ID : T170524-00101 / OP_INDEX : 423 / SECTION : 0 -  데이터 선택 (1)
SHIP_ID : T170524-00101 / OP_INDEX : 424 / SECTION : 0 -  데이터 선택 (1)
SHIP_ID : T170524-00101 / OP_INDEX

SHIP_ID : T170524-00101 / OP_INDEX : 509 / SECTION : 0 -  데이터 선택 (1)
SHIP_ID : T170524-00101 / OP_INDEX : 510 / SECTION : 0 -  데이터 선택 (1)
SHIP_ID : T170524-00101 / OP_INDEX : 511 / SECTION : 0 -  데이터 선택 (2)
SHIP_ID : T170524-00101 / OP_INDEX : 514 / SECTION : 0 -  데이터 선택 (15)
SHIP_ID : T170524-00101 / OP_INDEX : 515 / SECTION : 0 -  데이터 선택 (80)
SHIP_ID : T170524-00101 / OP_INDEX : 516 / SECTION : 0 -  데이터 선택 (42)
SHIP_ID : T170524-00101 / OP_INDEX : 517 / SECTION : 0 -  데이터 선택 (14)
SHIP_ID : T170524-00101 / OP_INDEX : 519 / SECTION : 0 -  데이터 선택 (268)
SHIP_ID : T170524-00101 / OP_INDEX : 519 / SECTION : 0 -  조건 통과
이상 값 제거 후 데이터가 비어 있습니다.
에러 발생: not enough values to unpack (expected 4, got 2). 다음 반복으로 넘어갑니다.
distribute_by_application 함수 실행 시간: 0.1060초
전처리 후 모델 데이터 프레임이 존재하지 않아 통계 알고리즘 단독 진행합니다.
Loaded data type: <class 'pycaret.internal.pipeline.Pipeline'>
 CSU 데이터 프레임:          SHIP_ID  OP_INDEX  SECTION  DATA_INDEX  HEALTH_SCORE
0  T170524-00101       519        0       137.0         

Loaded data type: <class 'pycaret.internal.pipeline.Pipeline'>
 CSU 데이터 프레임:          SHIP_ID  OP_INDEX  SECTION  DATA_INDEX  HEALTH_SCORE
0  T170713-01401      1867        0        96.0     73.244949
DataFrame has been successfully loaded into tc_ai_total_system_health_group_v0.0.0 table in ecs_test database.
apply_system_health_algorithms_with_total 함수 실행 시간: 0.6530초
SHIP_ID : T170713-01401 / OP_INDEX : 1868 / SECTION : 0 -  데이터 선택 (119)
SHIP_ID : T170713-01401 / OP_INDEX : 1869 / SECTION : 0 -  데이터 선택 (8)
SHIP_ID : T170713-01401 / OP_INDEX : 1870 / SECTION : 0 -  데이터 선택 (125)
SHIP_ID : T170713-01401 / OP_INDEX : 1871 / SECTION : 0 -  데이터 선택 (5)
SHIP_ID : T170713-01401 / OP_INDEX : 1873 / SECTION : 0 -  데이터 선택 (16)
SHIP_ID : T170713-01401 / OP_INDEX : 1874 / SECTION : 0 -  데이터 선택 (3)
SHIP_ID : T170713-01401 / OP_INDEX : 1875 / SECTION : 0 -  데이터 선택 (143)
SHIP_ID : T170713-01401 / OP_INDEX : 1876 / SECTION : 0 -  데이터 선택 (559)
SHIP_ID : T170713-01401 / OP_INDEX : 1876 / SECTION : 0 -  

SHIP_ID : T20180817002 / OP_INDEX : 666 / SECTION : 0 -  데이터 선택 (53)
SHIP_ID : T20180817002 / OP_INDEX : 667 / SECTION : 0 -  데이터 선택 (52)
SHIP_ID : T20180817002 / OP_INDEX : 668 / SECTION : 0 -  데이터 선택 (71)
SHIP_ID : T20180817002 / OP_INDEX : 669 / SECTION : 0 -  데이터 선택 (9)
SHIP_ID : T20180817002 / OP_INDEX : 670 / SECTION : 0 -  데이터 선택 (4)
SHIP_ID : T20180817003 / OP_INDEX : 435 / SECTION : 0 -  데이터 선택 (641)
SHIP_ID : T20180817003 / OP_INDEX : 435 / SECTION : 0 -  조건 통과
distribute_by_application 함수 실행 시간: 0.1109초
선박 데이터 프레임이 존재하지 않습니다.
SHIP_ID : T20180817003 / OP_INDEX : 436 / SECTION : 0 -  데이터 선택 (2)
SHIP_ID : T20180817003 / OP_INDEX : 437 / SECTION : 0 -  데이터 선택 (835)
SHIP_ID : T20180817003 / OP_INDEX : 437 / SECTION : 0 -  조건 통과
distribute_by_application 함수 실행 시간: 0.1030초
선박 데이터 프레임이 존재하지 않습니다.
SHIP_ID : T20180817003 / OP_INDEX : 438 / SECTION : 0 -  데이터 선택 (41)
SHIP_ID : T20180817003 / OP_INDEX : 439 / SECTION : 0 -  데이터 선택 (155)
SHIP_ID : T20180817003 / OP_INDEX : 440 / SECTION 

...

모델이 C:\Users\pc021\Desktop\dx_project\techross\HealthModelPipeline\dataflow\src\models_model\tro_group_model_v2.0.0에서 성공적으로 불러와졌습니다.
DataFrame has been successfully loaded into tc_ai_fault_group_v0.0.0 table in ecs_test database.
Database connection closed.
DataFrame has been successfully loaded into tc_ai_fault_model_group_v0.0.0 table in ecs_test database.
Database connection closed.
모델이 C:\Users\pc021\Desktop\dx_project\techross\HealthModelPipeline\dataflow\src\models_model\fts_model에서 성공적으로 불러와졌습니다.
DataFrame has been successfully loaded into tc_ai_fts_system_health_group_v0.0.0 table in ecs_test database.
Database connection closed.
DataFrame has been successfully loaded into tc_ai_fts_model_system_health_group_v0.0.0 table in ecs_test database.
Database connection closed.
모델이 C:\Users\pc021\Desktop\dx_project\techross\HealthModelPipeline\dataflow\src\models_model\fmu_model에서 성공적으로 불러와졌습니다.
DataFrame has been successfully loaded into tc_ai_fmu_system_health_group_v0.0.0 table in e

distribute_by_application 함수 실행 시간: 0.1223초
선박 데이터 프레임이 존재하지 않습니다.
SHIP_ID : T20230126002 / OP_INDEX : 196 / SECTION : 0 -  데이터 선택 (9)
SHIP_ID : T20230126002 / OP_INDEX : 197 / SECTION : 0 -  데이터 선택 (449)
SHIP_ID : T20230126002 / OP_INDEX : 197 / SECTION : 0 -  조건 통과
distribute_by_application 함수 실행 시간: 0.1232초
선박 데이터 프레임이 존재하지 않습니다.
SHIP_ID : T20230126002 / OP_INDEX : 198 / SECTION : 0 -  데이터 선택 (138)
SHIP_ID : T20230126002 / OP_INDEX : 199 / SECTION : 0 -  데이터 선택 (25)
SHIP_ID : T20230126002 / OP_INDEX : 200 / SECTION : 0 -  데이터 선택 (101)
SHIP_ID : T20230126002 / OP_INDEX : 201 / SECTION : 0 -  데이터 선택 (11)
SHIP_ID : T20230126002 / OP_INDEX : 202 / SECTION : 0 -  데이터 선택 (79)
SHIP_ID : T20230126002 / OP_INDEX : 203 / SECTION : 0 -  데이터 선택 (414)
SHIP_ID : T20230126002 / OP_INDEX : 203 / SECTION : 0 -  조건 통과
distribute_by_application 함수 실행 시간: 0.1130초
선박 데이터 프레임이 존재하지 않습니다.
SHIP_ID : T20230126002 / OP_INDEX : 204 / SECTION : 0 -  데이터 선택 (62)
SHIP_ID : T20230206001 / OP_INDEX : 54 / SECTION 

Loaded data type: <class 'pycaret.internal.pipeline.Pipeline'>
 CSU 데이터 프레임:         SHIP_ID  OP_INDEX  SECTION  DATA_INDEX  HEALTH_SCORE
0  T20230209001       872        0        96.5           100
DataFrame has been successfully loaded into tc_ai_total_system_health_group_v0.0.0 table in ecs_test database.
apply_system_health_algorithms_with_total 함수 실행 시간: 0.6315초
SHIP_ID : T20230209001 / OP_INDEX : 873 / SECTION : 0 -  데이터 선택 (9)
SHIP_ID : T20230209001 / OP_INDEX : 874 / SECTION : 0 -  데이터 선택 (87)
SHIP_ID : T20230209001 / OP_INDEX : 875 / SECTION : 0 -  데이터 선택 (3)
SHIP_ID : T20230209001 / OP_INDEX : 876 / SECTION : 0 -  데이터 선택 (8)
SHIP_ID : T20230209001 / OP_INDEX : 877 / SECTION : 0 -  데이터 선택 (27)
SHIP_ID : T20230209001 / OP_INDEX : 878 / SECTION : 0 -  데이터 선택 (6)
SHIP_ID : T20230209001 / OP_INDEX : 879 / SECTION : 0 -  데이터 선택 (131)
SHIP_ID : T20230209001 / OP_INDEX : 880 / SECTION : 0 -  데이터 선택 (2)
SHIP_ID : T20230209001 / OP_INDEX : 881 / SECTION : 0 -  데이터 선택 (6)
SHIP_ID : T202

distribute_by_application 함수 실행 시간: 0.1543초
선박 데이터 프레임이 존재하지 않습니다.
SHIP_ID : T20230417001 / OP_INDEX : 275 / SECTION : 0 -  데이터 선택 (33)
SHIP_ID : T20230511002 / OP_INDEX : 209 / SECTION : 0 -  데이터 선택 (2)
SHIP_ID : T20230511002 / OP_INDEX : 210 / SECTION : 0 -  데이터 선택 (6)
SHIP_ID : T20230511002 / OP_INDEX : 211 / SECTION : 0 -  데이터 선택 (6)
SHIP_ID : T20230511002 / OP_INDEX : 213 / SECTION : 0 -  데이터 선택 (5)
SHIP_ID : T20230511002 / OP_INDEX : 214 / SECTION : 0 -  데이터 선택 (82)
SHIP_ID : T20230511002 / OP_INDEX : 215 / SECTION : 0 -  데이터 선택 (40)
SHIP_ID : T20230511002 / OP_INDEX : 216 / SECTION : 0 -  데이터 선택 (5)
SHIP_ID : T20230511002 / OP_INDEX : 217 / SECTION : 0 -  데이터 선택 (6)
SHIP_ID : T20230511002 / OP_INDEX : 218 / SECTION : 0 -  데이터 선택 (27)
SHIP_ID : T20230511002 / OP_INDEX : 219 / SECTION : 0 -  데이터 선택 (92)
SHIP_ID : T20230511002 / OP_INDEX : 220 / SECTION : 0 -  데이터 선택 (114)
SHIP_ID : T20230511002 / OP_INDEX : 221 / SECTION : 0 -  데이터 선택 (74)
SHIP_ID : T20230511002 / OP_INDEX : 222 /

SHIP_ID : T99990000006 / OP_INDEX : 222 / SECTION : 0 -  데이터 선택 (14)
SHIP_ID : T99990000006 / OP_INDEX : 223 / SECTION : 0 -  데이터 선택 (19)
SHIP_ID : T99990000006 / OP_INDEX : 224 / SECTION : 0 -  데이터 선택 (1)
SHIP_ID : T99990000006 / OP_INDEX : 225 / SECTION : 0 -  데이터 선택 (5)
SHIP_ID : T99990000006 / OP_INDEX : 226 / SECTION : 0 -  데이터 선택 (17)
SHIP_ID : T99990000006 / OP_INDEX : 227 / SECTION : 0 -  데이터 선택 (7)
SHIP_ID : T99990000006 / OP_INDEX : 228 / SECTION : 0 -  데이터 선택 (3)
SHIP_ID : T99990000006 / OP_INDEX : 229 / SECTION : 0 -  데이터 선택 (20)
SHIP_ID : T99990000006 / OP_INDEX : 230 / SECTION : 0 -  데이터 선택 (15)
스케줄 시작
스케줄 시작
스케줄 시작
스케줄 시작
스케줄 시작
스케줄 시작
스케줄 시작
스케줄 시작
스케줄 시작
스케줄 시작
스케줄 시작
스케줄 시작
스케줄 시작
스케줄 시작
스케줄 시작
스케줄 시작
스케줄 시작
스케줄 시작
스케줄 시작
스케줄 시작
스케줄 시작
스케줄 시작
스케줄 시작
스케줄 시작
스케줄 시작
스케줄 시작
스케줄 시작
스케줄 시작
스케줄 시작
스케줄 시작
스케줄 시작
스케줄 시작
스케줄 시작
스케줄 시작
스케줄 시작
스케줄 시작
스케줄 시작
스케줄 시작
스케줄 시작
스케줄 시작
스케줄 시작
스케줄 시작
스케줄 시작
스케줄 시작
스케줄 시작
스케줄 시작
스케줄 시작
스케줄 시작
스케줄 시작
스케줄 시작
스케줄 시작
스케줄 시작
스케줄 시작
스케줄 시작
스케줄 시